In [1]:
import pandas as pd
from collections import Counter

In [2]:
df = pd.read_excel("data/newest_dataset_lxx_mt_dss.xlsx").fillna("")

In [3]:
pd.set_option("display.max_columns", False)
pd.set_option("display.max_rows", False)
pd.set_option("display.max_colwidth", False)

In [4]:
df.columns

Index(['level_0', 'verb_id', 'lex', 'scroll', 'book', 'chapter', 'verse_num',
       'gcons_verb', 'gcons_verse', 'sign_info', 'stem', 'tense',
       'gcons_clause', 'subject', 'complement', 'cmpl_lex', 'cmpl_translation',
       'dir_he', 'cmpl_constr', 'cmpl_nt', 'cmpl_anim', 'cmpl_det',
       'cmpl_indiv', 'cmpl_complex', 'motion_type', 'preposition_1',
       'preposition_2', 'preposition_3', 'preposition_4', 'comments',
       'reconstructed_verse', 'spatial_arg_type', 'index'],
      dtype='object')

In [5]:
stems = list(set(df.stem))
stems.remove("")
stems

['hof', 'qal', 'hif']

In [6]:
verbs = list(set(df.lex))
verbs

['ἐπάγω',
 'ἐξέρχομαι',
 'BW>[',
 'προσάγω ',
 'ἐπάγω',
 'φέρω',
 'ἀναφέρω',
 'προσφέρω',
 'τίθημι',
 'παραγίγνομαι',
 'εἰμί',
 'εἴσειμι',
 'συνάπτω',
 'εἰσάγω',
 'ἄγω',
 'ποιέω',
 'εἰσφέρω',
 'ἥκω',
 'δυσμή',
 'εἰσέρχομαι',
 'πορεύω',
 'εἰσπορεύω',
 'εἰσφέρω',
 'εἰσάγω',
 'προσέρχομαι',
 'ἔρχομαι']

In [7]:
list_paper1 = [
    'εἰσέρχομαι',
    'ἔρχομαι',
    'ἐξέρχομαι',
    'φέρω',
    'εἰσφέρω',
    'ἀναφέρω',
    'προσφέρω',
    'εἰσφέρω',
    'προσέρχομαι', 
 ]

In [8]:
{key:nb for (key, nb) in Counter(df.lex).items() if key in list_paper1}

{'εἰσέρχομαι': 32,
 'ἔρχομαι': 12,
 'ἐξέρχομαι': 2,
 'εἰσφέρω': 2,
 'εἰσφέρω': 6,
 'ἀναφέρω': 3,
 'προσέρχομαι': 1,
 'φέρω': 16,
 'προσφέρω': 1}

In [9]:
32 + 12 + 2 + 2 + 6 + 3 + 1 + 16 + 1

75

In [10]:
df_fero = df[df.lex == "φέρω"].copy()

In [11]:
len(list(df_fero.verb_id))

16

In [12]:
df_mt = df[df.scroll == "MT"]

In [13]:
df_qal = df_mt[df_mt.stem == "qal"]
len(df_qal)

80

In [14]:
len(set(df_qal.verb_id))

78

In [15]:
df_hiphil = df_mt[df_mt.stem == "hif"]
len(df_hiphil)

44

In [16]:
greek_verbs = list(set(df.lex))
greek_verbs.remove("BW>[")
greek_verbs

['ἐπάγω',
 'ἐξέρχομαι',
 'προσάγω ',
 'ἐπάγω',
 'φέρω',
 'ἀναφέρω',
 'προσφέρω',
 'τίθημι',
 'παραγίγνομαι',
 'εἰμί',
 'εἴσειμι',
 'συνάπτω',
 'εἰσάγω',
 'ἄγω',
 'ποιέω',
 'εἰσφέρω',
 'ἥκω',
 'δυσμή',
 'εἰσέρχομαι',
 'πορεύω',
 'εἰσπορεύω',
 'εἰσφέρω',
 'εἰσάγω',
 'προσέρχομαι',
 'ἔρχομαι']

In [17]:
short_df = df.head(10)
short_df
short_df.to_csv("short_dataset.csv", index=False)

In [18]:
from collections import defaultdict

# Filter LXX and MT rows
df_lxx = df[df['scroll'] == 'LXX']
df_mt = df[df['scroll'] == 'MT']

# Dictionary to store outputs organized by Greek verb (lex)
output_dict = defaultdict(list)

# Iterate over LXX verbs
for _, row in df_lxx.iterrows():
    greek_number = row['verb_id']
    greek_verb = row['lex']
    greek_verse = row['gcons_verse']
    greek_clause = row["gcons_clause"]
    greek_chapter = row['chapter']
    greek_verse_num = row['verse_num']
    
    # Collect all non-null Greek prepositions
    greek_prepositions = [row[col] for col in ['preposition_1', 'preposition_2', 'preposition_3', 'preposition_4'] if pd.notna(row[col])]
    greek_prep_str = ', '.join(greek_prepositions) if greek_prepositions else 'no preposition'
    
    # Find corresponding Hebrew verb by verb_id (replace _lxx with _mt)
    hebrew_verb_id = row['verb_id'].replace('_lxx', '_mt')
    hebrew_row = df_mt[df_mt['verb_id'] == hebrew_verb_id]

    if not hebrew_row.empty:
        hebrew_row = hebrew_row.iloc[0]
        hebrew_number = hebrew_row['verb_id']
        hebrew_verse = hebrew_row['gcons_verse']
        hebrew_complement = hebrew_row["complement"]
        cmpl_constr = hebrew_row['cmpl_constr'] if pd.notna(hebrew_row['cmpl_constr']) else 'no complement'
        hebrew_prepositions = [hebrew_row[col] for col in ['preposition_1', 'preposition_2', 'preposition_3', 'preposition_4'] if pd.notna(hebrew_row[col])]
        hebrew_prep_str = ', '.join(hebrew_prepositions) if hebrew_prepositions else 'no preposition'
        hebrew_animacy = hebrew_row['cmpl_anim']
    else:
        hebrew_verse = 'no matching Hebrew verse'
        cmpl_constr = 'no complement'
        hebrew_prep_str = 'no preposition'

    # Store the results
    output_dict[greek_verb].append({
        'Greek verb id': greek_number,
        'Hebrew verb id': hebrew_number,
        'Greek Verse': greek_verse,
        'Greek Clause': greek_clause,
        'Greek Locus': (greek_chapter, greek_verse_num),
        'Greek Preposition(s)': greek_prep_str.replace(',', '').replace(' ', ''),
        'Hebrew Verse': hebrew_verse,
        'Hebrew Complement': hebrew_complement,
        'Hebrew Construction': cmpl_constr,
        'Hebrew Preposition(s)': hebrew_prep_str.replace(',', '').replace(' ', ''),
        'Hebrew Complement Animacy': hebrew_animacy,
    })

# Convert to displayable structure
import pandas as pd
from IPython.display import display

# Create a multi-section display for each Greek verb
for greek_verb, entries in output_dict.items():
    print(f"\n--- Greek verb: {greek_verb} ---")
    display(pd.DataFrame(entries))


--- Greek verb: εἰσπορεύω ---


,Greek verb id,Hebrew verb id,Greek Verse,Greek Clause,Greek Locus,Greek Preposition(s),Hebrew Verse,Hebrew Complement,Hebrew Construction,Hebrew Preposition(s),Hebrew Complement Animacy
0,28771_lxx,28771_mt,"ταῦτα τὰ ὀνόματα τῶν υἱῶν Ἰσραὴλ εἰσπεπορευμένων εἰς Αἴγυπτον ἅμα Ἰακὼβ τῷ πατρὶ αὐτῶν, ἕκαστος πανοικὶ αὐτῶν εἰσήλθοσαν.",,"(1, 1)",εἰς,W >LH CMWT BNJ JFR>L H B>JM MYRJMH >T J<QB >JC W BJTW B>W,MYRJMH,dir_he,,inanim
1,36834_lxx,36834_mt,"καὶ ἐπαναστραφὲν τὸ ὕδωρ ἐκάλυψε τὰ ἅρματα καὶ τοὺς ἀναβάτας καὶ πᾶσαν τὴν δύναμιν Φαραώ, τοὺς εἰσπορευομένους ὀπίσω αὐτῶν, εἰς τὴν θάλασσαν, καὶ οὐ κατελήφθη ἐξ αὐτῶν οὐ -δὲ εἷς.",,"(14, 28)",ὀπίσω,W JCBW H MJM W JKSW >T H RKB W >T H PRCJM L KL XJL PR<H H B>JM >XRJHM B JM L> NC>R BHM <D >XD,>XRJHM,prep,>XR/,anim
2,36834_lxx,36834_mt,"καὶ ἐπαναστραφὲν τὸ ὕδωρ ἐκάλυψε τὰ ἅρματα καὶ τοὺς ἀναβάτας καὶ πᾶσαν τὴν δύναμιν Φαραώ, τοὺς εἰσπορευομένους ὀπίσω αὐτῶν, εἰς τὴν θάλασσαν, καὶ οὐ κατελήφθη ἐξ αὐτῶν οὐ -δὲ εἷς.",,"(14, 28)",εἰς,W JCBW H MJM W JKSW >T H RKB W >T H PRCJM L KL XJL PR<H H B>JM >XRJHM B JM L> NC>R BHM <D >XD,>XRJHM,prep,>XR/,anim
3,41487_lxx,41487_mt,"καὶ τὸν φόβον ἀποστελῶ ἡγούμενόν σου καὶ ἐκστήσω πάντα τά ἔθνη, εἰς οὓς σὺ εἰσπορεύῃ εἰς αὐτούς, καὶ δώσω πάντας τούς ὑπεναντίους σου φυγάδας.",,"(23, 27)",εἰς,>T >JMTJ >CLX L PNJK W HMTJ >T KL H <M >CR TB> BHM W NTTJ >T KL >JBJK >LJK <RP,BHM,prep,B,anim
4,44272_lxx,44272_mt,καὶ ἐπιθήσεις ἐπὶ τὸ λογεῖον τῆς κρίσεως τὴν δήλωσιν καὶ τὴν ἀλήθειαν καὶ ἔσται ἐπὶ τοῦ στήθους Ααρων ὅταν εἰσπορεύηται εἰς τὸ ἅγιον ἐναντίον κυρίου καὶ οἴσει Ααρων τὰς κρίσεις τῶν υἱῶν Ισραηλ ἐπὶ τοῦ στήθους ἐναντίον κυρίου διὰ παντός,,"(28, 30)",εἰς,W NTT >L XCN H MCPV >T H >WRJM W >T H TMJM W HJW <L LB >HRN B B>W L PNJ JHWH W NF> >HRN >T MCPV BNJ JFR>L <L LBW L PNJ JHWH TMJD,L PNJ JHWH,prep,L,inanim
5,44513_lxx,44513_mt,καὶ ἕξει Ααρων αὐτὰ καὶ οἱ υἱοὶ αὐτοῦ ὡς ἂν εἰσπορεύωνται εἰς τὴν σκηνὴν τοῦ μαρτυρίου ἢ ὅταν προσπορεύωνται λειτουργεῖν πρὸς τὸ θυσιαστήριον τοῦ ἁγίου καὶ οὐκ ἐπάξονται πρὸς ἑαυτοὺς ἁμαρτίαν ἵνα μὴ ἀποθάνωσιν νόμιμον αἰώνιον αὐτῷ καὶ τῷ σπέρματι αὐτοῦ μετ᾽ αὐτόν,,"(28, 43)",εἰς,W HJW <L >HRN W <L BNJW B B>M >L >HL MW<D >W B GCTM >L H MZBX L CRT B QDC W L> JF>W <WN W MTW XQT <WLM LW W L ZR<W >XRJW,>L >HL MW<D,prep,>L,inanim
6,45868_lxx,45868_mt,"ὅταν εἰσπορεύωνται εἰς τὴν σκηνὴν τοῦ μαρτυρίου, νίψονται ὕδατι καὶ οὐ μὴ ἀποθάνωσιν · ἢ ὅταν προσπορεύωνται πρὸς τὸ θυσιαστήριον λειτουργεῖν καὶ ἀναφέρειν τὰ ὁλοκαυτώματα Κυρίῳ,",,"(30, 20)",εἰς,B B>M >L >HL MW<D JRXYW MJM W L> JMTW >W B GCTM >L H MZBX L CRT L HQVJR >CH L JHWH,>L >HL MW<D,prep,>L,inanim
7,48059_lxx,48059_mt,"πρόσεχε σεαυτῷ, μή ποτε θῇς διαθήκην τοῖς ἐγκαθημένοις ἐπὶ τῆς γῆς, εἰς ἣν εἰσπορεύῃ εἰς αὐτήν, μή σοι γένηται πρόσκομμα ἐν ὑμῖν.",,"(34, 12)",εἰς,HCMR LK PN TKRT BRJT L JWCB H >RY >CR >TH B> <LJH PN JHJH L MWQC B QRBK,<LJH,prep,<L,inanim
8,48453_lxx,48453_mt,"ἡνίκα δ’ ἂν εἰσεπορεύετο Μωυσῆς ἔναντι Κυρίου λαλεῖν αὐτῷ, περιῃρεῖτο τὸ κάλυμμα ἕως τοῦ ἐκπορεύεσθαι. καὶ ἐξελθὼν ἐλάλει πᾶσι τοῖς υἱοῖς Ἰσραὴλ ὅσα ἐνετείλατο αὐτῷ Κύριος,",,"(34, 34)",ἔναντι,W B B> MCH L PNJ JHWH L DBR >TW JSJR >T H MSWH <D Y>TW W JY> W DBR >L BNJ JFR>L >T >CR JYWH,L PNJ JHWH,prep,L,inanim
9,52388_lxx,52388_mt,καὶ ἐποίησεν τὸν λουτῆρα ἵνα νίπτωνται ἐξ αὐτοῦ Μωυσῆς καὶ Ααρων καὶ οἱ υἱοὶ αὐτοῦ τὰς χεῖρας αὐτῶν καὶ τοὺς πόδας εἰσπορευομένων αὐτῶν εἰς τὴν σκηνὴν τοῦ μαρτυρίου ἢ ὅταν προσπορεύωνται πρὸς τὸ θυσιαστήριον λειτουργεῖν ἐνίπτοντο ἐξ αὐτοῦ καθάπερ συνέταξεν κύριος τῷ Μωυσῇ,,"(38, 27)",εἰς,B B>M >L >HL MW<D W B QRBTM >L H MZBX JRXYW K >CR YWH JHWH >T MCH,>L >HL MW<D,prep,>L,inanim



--- Greek verb: εἰσέρχομαι ---


,Greek verb id,Hebrew verb id,Greek Verse,Greek Clause,Greek Locus,Greek Preposition(s),Hebrew Verse,Hebrew Complement,Hebrew Construction,Hebrew Preposition(s),Hebrew Complement Animacy
0,28778_lxx,28778_mt,"ταῦτα τὰ ὀνόματα τῶν υἱῶν Ἰσραὴλ εἰσπεπορευμένων εἰς Αἴγυπτον ἅμα Ἰακὼβ τῷ πατρὶ αὐτῶν, ἕκαστος πανοικὶ αὐτῶν εἰσήλθοσαν.",,"(1, 1)",,W >LH CMWT BNJ JFR>L H B>JM MYRJMH >T J<QB >JC W BJTW B>W,no complement,,,
1,29061_lxx,29061_mt,"εἶπαν δὲ αἱ μαῖαι τῷ Φαραώ · οὐχ ὡς γυναῖκες Αἰγύπτου αἱ Ἑβραῖαι, τίκτουσι γὰρ πρὶν ἢ εἰσελθεῖν πρὸς αὐτὰς τὰς μαίας · E καὶ ἔτικτον.",,"(1, 19)",πρός,W T>MRN H MJLDT >L PR<H KJ L> K NCJM H MYRJT H <BRJT KJ XJWT HNH B VRM TBW> >LHN H MJLDT W JLDW,>LHN,prep,>L,anim
2,30056_lxx,30056_mt,"καὶ εἰσακούσονταί σου τῆς φωνῆς · καὶ εἰσελεύσῃ σὺ καὶ ἡ γερουσία Ἰσραὴλ πρὸς Φαραὼ βασιλέα Αἰγύπτου καὶ ἐρεῖς πρὸς αὐτόν · ὁ Θεὸς τῶν Ἑβραίων προσκέκληται ἡμᾶς · πορευσόμεθα οὖν ὁδὸν τριῶν ἡμερῶν εἰς τὴν ἔρημον, ἵνα θύσωμεν τῷ Θεῷ ἡμῶν.",,"(3, 18)",πρός,W CM<W L QLK W B>T >TH W ZQNJ JFR>L >L MLK MYRJM W >MRTM >LJW JHWH >LHJ H <BRJJM NQRH <LJNW W <TH NLKH N> DRK CLCT JMJM B MDBR W NZBXH L JHWH >LHJNW,>L MLK MYRJM,prep,>L,inanim
3,30810_lxx,30810_mt,"καὶ μετὰ ταῦτα εἰσῆλθε Μωυσῆς καὶ Ἀαρὼν πρὸς Φαραὼ καὶ εἶπαν αὐτῷ · τάδε λέγει Κύριος ὁ Θεὸς Ἰσραήλ · ἐξαπόστειλον τὸν λαόν μου, ἵνα μοι ἑορτάσωσιν ἐν τῇ ἐρήμῳ.",,"(5, 1)",πρός,W >XR B>W MCH W >HRN W J>MRW >L PR<H KH >MR JHWH >LHJ JFR>L CLX >T <MJ W JXGW LJ B MDBR,no complement,,,
4,31093_lxx,31093_mt,εἰσελθόντες δὲ οἱ γραμματεῖς τῶν υἱῶν Ἰσραὴλ κατεβόησαν πρὸς Φαραὼ λέγοντες · ἱνατί σὺ οὕτως ποιεῖς τοῖς σοῖς οἰκέταις;,,"(5, 15)",,W JB>W CVRJ BNJ JFR>L W JY<QW >L PR<H L >MR LMH T<FH KH L <BDJK,no complement,,,
5,31441_lxx,31441_mt,"εἴσελθε, λάλησον Φαραὼ βασιλεῖ Αἰγύπτου, ἵνα ἐξαποστείλῃ τοὺς υἱοὺς Ἰσραὴλ ἐκ τῆς γῆς αὐτοῦ.",,"(6, 11)",,B> DBR >L PR<H MLK MYRJM W JCLX >T BNJ JFR>L M >RYW,no complement,,,
6,31942_lxx,31942_mt,"εἰσῆλθε δὲ Μωυσῆς καὶ Ἀαρὼν ἐναντίον Φαραὼ καὶ τῶν θεραπόντων αὐτοῦ καὶ ἐποίησαν οὕτως, καθάπερ ἐνετείλατο αὐτοῖς Κύριος · καὶ ἔρριψεν Ἀαρὼν τὴν ράβδον ἐναντίον Φαραώ, καὶ ἐναντίον τῶν θεραπόντων αὐτοῦ, καὶ ἐγένετο δράκων.",,"(7, 10)",ἐναντίον,W JB> MCH W >HRN >L PR<H W J<FW KN K >CR YWH JHWH W JCLK >HRN >T MVHW L PNJ PR<H W L PNJ <BDJW W JHJ L TNJN,>L PR<H,prep,>L,anim
7,32274_lxx,32274_mt,ἐπιστραφεὶς δὲ Φαραὼ εἰσῆλθεν εἰς τὸν οἶκον αὐτοῦ καὶ οὐκ ἐπέστησε τὸν νοῦν αὐτοῦ οὐ -δὲ ἐπὶ τούτῳ.,,"(7, 23)",εἰς,W JPN PR<H W JB> >L BJTW W L> CT LBW GM L Z>T,>L BJTW,prep,>L,inanim
8,32318_lxx,32318_mt,"εἶπε δὲ Κύριος πρὸς Μωυσῆν · εἴσελθε πρὸς Φαραὼ καὶ ἐρεῖς πρὸς αὐτόν · τάδε λέγει Κύριος · ἐξαπόστειλον τὸν λαόν μου, ἵνα μοι λατρεύσωσιν ·",,"(7, 26)",πρός,W J>MR JHWH >L MCH B> >L PR<H W >MRT >LJW KH >MR JHWH CLX >T <MJ W J<BDNJ,>L PR<H,prep,>L,anim
9,32355_lxx,32355_mt,"καὶ ἐξερεύξεται ὁ ποταμὸς βατράχους, καὶ ἀναβάντες εἰσελεύσονται εἰς τοὺς οἴκους σου καὶ εἰς τὰ ταμιεῖα τῶν κοιτώνων σου καὶ ἐπὶ τῶν κλινῶν σου καὶ ἐπὶ τοὺς οἴκους τῶν θεραπόντων σου καὶ τοῦ λαοῦ σου καὶ ἐν τοῖς φυράμασί σου καὶ ἐν τοῖς κλιβάνοις σου ·",,"(7, 28)",εἰς,W CRY H J>R YPRD<JM W <LW W B>W B BJTK W B XDR MCKBK W <L MVTK W B BJT <BDJK W B <MK W B TNWRJK W B MC>RWTJK,B BJTK W B XDR MCKBK W <L MVTK W B BJT <BDJK W B <MK W B TNWRJK W B MC>RWTJK,prep + prep,BB<LB,inanim



--- Greek verb: εἰσάγω ---


...

,Greek verb id,Hebrew verb id,Greek Verse,Greek Clause,Greek Locus,Greek Preposition(s),Hebrew Verse,Hebrew Complement,Hebrew Construction,Hebrew Preposition(s),Hebrew Complement Animacy
0,48288_lxx,48288_mt,τὰ πρωτογεννήματα τῆς γῆς σου θήσεις εἰς τὸν οἶκον Κυρίου τοῦ Θεοῦ σου. οὐχ ἑψήσεις ἄρνα ἐν γάλακτι μητρὸς αὐτοῦ.,,"(34, 26)",εἰς,R>CJT BKWRJ >DMTK TBJ> BJT JHWH >LHJK L> TBCL GDJ B XLB >MW,BJT JHWH >LHJK,vc,,inanim



--- Greek verb: προσφέρω ---


,Greek verb id,Hebrew verb id,Greek Verse,Greek Clause,Greek Locus,Greek Preposition(s),Hebrew Verse,Hebrew Complement,Hebrew Construction,Hebrew Preposition(s),Hebrew Complement Animacy
0,49331_lxx,49331_mt,καὶ προσέταξε Μωυσῆς καὶ ἐκήρυξεν ἐν τῇ παρεμβολῇ λέγων · ἀνὴρ καὶ γυνὴ μηκέτι ἐργαζέσθωσαν εἰς τὰς ἀπαρχὰς τοῦ ἁγίου · καὶ ἐκωλύθη ὁ λαὸς ἔτι προσφέρειν.,,"(36, 6)",,W JYW MCH W J<BJRW QWL B MXNH L >MR >JC W >CH >L J<FW <WD ML>KH L TRWMT H QDC W JKL> H <M M HBJ>,no complement,,,



--- Greek verb: ποιέω ---


,Greek verb id,Hebrew verb id,Greek Verse,Greek Clause,Greek Locus,Greek Preposition(s),Hebrew Verse,Hebrew Complement,Hebrew Construction,Hebrew Preposition(s),Hebrew Complement Animacy
0,,52450_mt,οὗτος ἐποίησεν τῷ θυσιαστηρίῳ παράθεμα ἔργον δικτυωτόν κάτωθεν τοῦ πυρείου ὑπὸ αὐτὸ ἕως τοῦ ἡμίσους αὐτοῦ καὶ ἐπέθηκεν αὐτῷ τέσσαρας δακτυλίους ἐκ τῶν τεσσάρων μερῶν τοῦ παραθέματος τοῦ θυσιαστηρίου χαλκοῦς τοῖς μοχλοῖς εὐρεῖς ὥστε αἴρειν τὸ θυσιαστήριον ἐν αὐτοῖς,,"(38, 24)",,no matching Hebrew verse,>L >HL MW<D,no complement,nopreposition,inanim


In [19]:
import os

# Create an output directory if it doesn't exist
output_dir = 'data/greek_verbs'
os.makedirs(output_dir, exist_ok=True)

# Save each Greek verb's data to a separate Excel file
for greek_verb, entries in output_dict.items():
    # Clean the filename to be safe for file systems
    safe_verb = ''.join(c if c.isalnum() else '_' for c in greek_verb)
    filename = f"{safe_verb}.xlsx"
    filepath = os.path.join(output_dir, filename)

    # Save to Excel
    df_output = pd.DataFrame(entries)
    df_output.to_excel(filepath, index=False)

# Return path to output directory for user access
output_dir

'data/greek_verbs'

In [20]:
greek_lex_stems = {}

for stem in stems:
    heb_verb_ids = df[df.stem == stem].verb_id
    greek_verb_ids = [verb_id.replace("mt", "lxx") for verb_id in heb_verb_ids]
    df_greek_ids = df[df.verb_id.isin(greek_verb_ids)]
    lexemes = list(df_greek_ids.lex)
    prepositions = list(df_greek_ids.preposition_1)
    loci = list(zip(list(df_greek_ids.chapter), list(df_greek_ids.verse_num)))

    lex_dict = {}

    for lex, prep, locus in zip(lexemes, prepositions, loci):
        if lex not in lex_dict:
            lex_dict[lex] = {}
        if prep not in lex_dict[lex]:
            lex_dict[lex][prep] = []
        lex_dict[lex][prep].append(locus)

    for lex, sub_dict in lex_dict.items():
        new_dict = {}
        for prep, loci in sub_dict.items():
            if prep == "":
                prep = "no prep in Greek"
            new_dict[f"{prep}, {len(loci)} occ."] = loci
        lex_dict[lex] = new_dict

    
    greek_lex_stems[stem] = lex_dict

In [21]:
greek_lex_stems

{'hof': {'εἰσάγω': {'εἰς, 1 occ.': [(27, 7)]}},
 'qal': {'εἰσπορεύω': {'εἰς, 8 occ.': [(1, 1),
    (14, 28),
    (23, 27),
    (28, 30),
    (28, 43),
    (30, 20),
    (34, 12),
    (38, 27)],
   'ὀπίσω, 1 occ.': [(14, 28)],
   'ἔναντι, 1 occ.': [(34, 34)]},
  'εἰσέρχομαι': {'no prep in Greek, 8 occ.': [(1, 1),
    (5, 15),
    (6, 11),
    (15, 19),
    (16, 22),
    (21, 3),
    (24, 3),
    (34, 35)],
   'πρός, 6 occ.': [(1, 19), (3, 18), (5, 1), (7, 26), (9, 1), (10, 1)],
   'ἐναντίον, 2 occ.': [(7, 10), (10, 3)],
   'εἰς, 10 occ.': [(7, 23),
    (7, 28),
    (12, 23),
    (12, 25),
    (14, 16),
    (24, 18),
    (29, 30),
    (33, 8),
    (33, 9),
    (40, 35)],
   'ὀπίσω, 2 occ.': [(14, 17), (14, 23)],
   'ἀνὰ μέσον, 1 occ.': [(14, 20)],
   'εἰς μέσον, 2 occ.': [(14, 22), (14, 23)]},
  'BW>[': {'no prep in Greek, 3 occ.': [(1, 1), (5, 1), (15, 23)],
   '>L, 10 occ.': [(1, 19),
    (3, 9),
    (3, 13),
    (10, 1),
    (14, 23),
    (14, 23),
    (18, 5),
    (18, 

In [22]:
heb_preps = list(set(df[df.scroll == "MT"].preposition_1))
#preps.remove("")
heb_preps

['', '<L', 'MN', 'BJN/', 'B', 'L', '>L', '<D', '>XR/']

In [23]:
greek_lex_preps = {}


for heb_prep in heb_preps:
    heb_verb_ids = df[df.preposition_1 == heb_prep].verb_id
    greek_verb_ids = [verb_id.replace("mt", "lxx") for verb_id in heb_verb_ids]
    df_greek_ids = df[df.verb_id.isin(greek_verb_ids)]
    lexemes = list(df_greek_ids.lex)
    prepositions = list(df_greek_ids.preposition_1)
    loci = list(zip(list(df_greek_ids.chapter), list(df_greek_ids.verse_num)))

    lex_dict = {}

    for lex, prep, locus in zip(lexemes, prepositions, loci):
        if lex not in lex_dict:
            lex_dict[lex] = {}
        if prep not in lex_dict[lex]:
            lex_dict[lex][prep] = []
        lex_dict[lex][prep].append(locus)

    for lex, sub_dict in lex_dict.items():
        new_dict = {}
        for prep, loci in sub_dict.items():
            if prep == "":
                prep = "no prep in Greek"
            new_dict[f"{prep}, {len(loci)} occ."] = loci
        lex_dict[lex] = new_dict

    
    greek_lex_preps[heb_prep] = lex_dict

In [24]:
greek_lex_preps

{'': {'εἰσπορεύω': {'εἰς, 1 occ.': [(1, 1)]},
  'εἰσέρχομαι': {'no prep in Greek, 9 occ.': [(1, 1),
    (5, 15),
    (6, 11),
    (15, 19),
    (16, 22),
    (21, 3),
    (24, 3),
    (34, 35),
    (35, 29)],
   'πρός, 1 occ.': [(5, 1)],
   'εἰς, 2 occ.': [(33, 8), (33, 9)]},
  'BW>[': {'no prep in Greek, 7 occ.': [(1, 1),
    (5, 1),
    (15, 17),
    (15, 23),
    (18, 7),
    (26, 33),
    (40, 21)]},
  'παραγίγνομαι': {'no prep in Greek, 5 occ.': [(2, 16),
    (2, 17),
    (2, 18),
    (18, 12),
    (36, 4)],
   'πρός, 1 occ.': [(20, 20)]},
  'ἐπάγω': {'ἐπὶ, 1 occ.': [(10, 4)]},
  'ἔρχομαι': {'no prep in Greek, 4 occ.': [(10, 26),
    (17, 8),
    (19, 7),
    (35, 10)],
   'εἰς, 4 occ.': [(15, 23), (15, 27), (19, 1), (19, 2)]},
  'εἰσάγω': {'εἰς, 1 occ.': [(15, 17)]},
  'εἰσφέρω': {'no prep in Greek, 3 occ.': [(16, 5), (40, 4), (40, 4)],
   'εἰς, 1 occ.': [(23, 19)],
   'ἐκεῖ ἐσώτερον, 1 occ.': [(26, 33)]},
  'δυσμή': {'no prep in Greek, 2 occ.': [(17, 12), (22, 25)]},
 

In [25]:
df[(df.lex == "εἰσέρχομαι") & (df.preposition_1 == "")]

,level_0,verb_id,lex,scroll,book,chapter,verse_num,gcons_verb,gcons_verse,sign_info,stem,tense,gcons_clause,subject,complement,cmpl_lex,cmpl_translation,dir_he,cmpl_constr,cmpl_nt,cmpl_anim,cmpl_det,cmpl_indiv,cmpl_complex,motion_type,preposition_1,preposition_2,preposition_3,preposition_4,comments,reconstructed_verse,spatial_arg_type,index
3,3,28778_lxx,εἰσέρχομαι,LXX,Exodus,1,1,εἰσήλθοσαν,"ταῦτα τὰ ὀνόματα τῶν υἱῶν Ἰσραὴλ εἰσπεπορευμένων εἰς Αἴγυπτον ἅμα Ἰακὼβ τῷ πατρὶ αὐτῶν, ἕκαστος πανοικὶ αὐτῶν εἰσήλθοσαν.",,,aor,,(grammatically no separate subject)no complement,no complement,no complement,,,,,,,,,,,,,,,,,
37,37,31093_lxx,εἰσέρχομαι,LXX,Exodus,5,15,εἰσελθόντες,εἰσελθόντες δὲ οἱ γραμματεῖς τῶν υἱῶν Ἰσραὴλ κατεβόησαν πρὸς Φαραὼ λέγοντες · ἱνατί σὺ οὕτως ποιεῖς τοῖς σοῖς οἰκέταις;,,,aor. Part.,,οἱ γραμματεῖς τῶν υἱῶν Ἰσραὴλ,no complement,,,,,,,,,,,,,,,,,,
45,45,31441_lxx,εἰσέρχομαι,LXX,Exodus,6,11,εἴσελθε,"εἴσελθε, λάλησον Φαραὼ βασιλεῖ Αἰγύπτου, ἵνα ἐξαποστείλῃ τοὺς υἱοὺς Ἰσραὴλ ἐκ τῆς γῆς αὐτοῦ.",,,aor. Imperat.,,,no complement,no complement,,,,,,,,,,,,,,,,,
117,117,37144_lxx,εἰσέρχομαι,LXX,Exodus,15,19,εἰσῆλθεν,"ὅτι εἰσῆλθεν ἵππος Φαραὼ σὺν ἅρμασι καὶ ἀναβάταις εἰς θάλασσαν, καὶ ἐπήγαγεν ἐπ’ αὐτοὺς Κύριος τὸ ὕδωρ τῆς θαλάσσης · οἱ δὲ υἱοὶ Ἰσραὴλ ἐπορεύθησαν διὰ ξηρᾶς ἐν μέσῳ τῆς θαλάσσης.",,,aor.,,ἵππος Φαραω,no complement (to the sea is the complement of the next verb),,,,,,,,,,,,,,,,,,
128,128,37837_lxx,εἰσέρχομαι,LXX,Exodus,16,22,εἰσήλθοσαν,"ἐγένετο δὲ τῇ ἡμέρᾳ τῇ ἕκτῃ, συνέλεξαν τὰ δέοντα διπλᾶ, δύο γομὸρ τῷ ἑνί · εἰσήλθοσαν δὲ πάντες οἱ ἄρχοντες τῆς συναγωγῆς καὶ ἀνήγγειλαν Μωυσῇ ·",,,aor.,,πάντες οἱ ἄρχοντες τῆς συναγωγῆς,no complement,no complement,,,,,,,,,,,,,,,,,
177,177,40043_lxx,εἰσέρχομαι,LXX,Exodus,21,3,εἰσέλθῃ,"ἐὰν αὐτὸς μόνος εἰσέλθῃ, καὶ μόνος ἐξελεύσεται · ἐὰν δὲ γυνὴ συνεισέλθῃ μετ’ αὐτοῦ, ἐξελεύσεται καὶ ἡ γυνὴ αὐτοῦ.",,,aor. Konj.,,,no complement,no complement,,,,,,,,,,,,,,,,,
195,195,41642_lxx,εἰσέρχομαι,LXX,Exodus,24,3,εἰσῆλθε,"εἰσῆλθε δὲ Μωυσῆς καὶ διηγήσατο τῷ λαῷ πάντα τὰ ρήματα τοῦ Θεοῦ καὶ τὰ δικαιώματα · ἀπεκρίθη δὲ πᾶς ὁ λαὸς φωνῇ μιᾷ λέγοντες · πάντας τοὺς λόγους, οὓς ἐλάλησε Κύριος, ποιήσομεν καὶ ἀκουσόμεθα.",,,aor.,,Μωυσῆς,no complement,no complement,,,,,,,,,,,,,,,,,
236,236,48498_lxx,εἰσέρχομαι,LXX,Exodus,34,35,εἰσέλθῃ,"καὶ εἶδον οἱ υἱοὶ Ἰσραὴλ τὸ πρόσωπον Μωυσέως ὅτι δεδόξασται, καὶ περιέθηκε Μωυσῆς κάλυμμα ἐπὶ τὸ πρόσωπον ἑαυτοῦ, ἕως ἂν εἰσέλθῃ συλλαλεῖν αὐτῷ.",,,aor. Konj.,,,no complement,no complement,,,,,,,,,,,,,,,,,
261,261,49025_lxx,εἰσέρχομαι,LXX,Exodus,35,29,εἰσελθόντας,"καὶ πᾶς ἀνὴρ καὶ γυνή, ὧν ἔφερεν ἡ διάνοια αὐτῶν εἰσελθόντας ποιεῖν πάντα τὰ ἔργα, ὅσα συνέταξε Κύριος ποιῆσαι αὐτὰ διά Μωυσῆ, ἤνεγκαν οἱ υἱοὶ Ἰσραὴλ ἀφαίρεμα Κυρίῳ.",,,aor. Part.,,,no complement (only inf. as the direct object),no complement,,,,,,,,,,,,,,,,,


In [26]:
df[(df.chapter == 1) & (df.verse_num == 1)]

,level_0,verb_id,lex,scroll,book,chapter,verse_num,gcons_verb,gcons_verse,sign_info,stem,tense,gcons_clause,subject,complement,cmpl_lex,cmpl_translation,dir_he,cmpl_constr,cmpl_nt,cmpl_anim,cmpl_det,cmpl_indiv,cmpl_complex,motion_type,preposition_1,preposition_2,preposition_3,preposition_4,comments,reconstructed_verse,spatial_arg_type,index
0,0,28771_mt,BW>[,MT,Exodus,1,1,B>JM,W >LH CMWT BNJ JFR>L H B>JM MYRJMH >T J<QB >JC W BJTW B>W,,qal,ptca,H B>JM MYRJMH >T J<QB,,MYRJMH,MYRJM/,to Egypt,1.0,dir_he,topo,inanim,det,nmpr,simple,factive,,,,,,,goal,
1,1,28778_mt,BW>[,MT,Exodus,1,1,B>W,W >LH CMWT BNJ JFR>L H B>JM MYRJMH >T J<QB >JC W BJTW B>W,,qal,perf,>JC W BJTW B>W,>JC W BJTW,no complement,no complement,,,,,,,,,,,,,,,,,
2,2,28771_lxx,εἰσπορεύω,LXX,Exodus,1,1,εἰσπεπορευμένων,"ταῦτα τὰ ὀνόματα τῶν υἱῶν Ἰσραὴλ εἰσπεπορευμένων εἰς Αἴγυπτον ἅμα Ἰακὼβ τῷ πατρὶ αὐτῶν, ἕκαστος πανοικὶ αὐτῶν εἰσήλθοσαν.",,,Perf. Pass. Part. Pl. Gen. Abs.,,,εἰς Αἴγυπτον,εἰς / Αἴγυπτος,to Egypt,,,,,,,,,εἰς,,,,this is a rare verb!,,,
3,3,28778_lxx,εἰσέρχομαι,LXX,Exodus,1,1,εἰσήλθοσαν,"ταῦτα τὰ ὀνόματα τῶν υἱῶν Ἰσραὴλ εἰσπεπορευμένων εἰς Αἴγυπτον ἅμα Ἰακὼβ τῷ πατρὶ αὐτῶν, ἕκαστος πανοικὶ αὐτῶν εἰσήλθοσαν.",,,aor,,(grammatically no separate subject)no complement,no complement,no complement,,,,,,,,,,,,,,,,,
4,4,1941992_dss,BW>[,4Q11,Exodus,1,1,B>W,>LH CMWT BNJ JFR>L H B>JM MYRJMH >T J<QB >JC W BJTW B>W,000 0000 000 00111 1 1111 111000 11 1111 111 0 0000 000,qal,perf,>JC W BJTW B>W,>JC W BJTW,no complement,no complement,,,,,,,,,,,,,,,>LH CMWT BNJ JF[R>L H B>JM MYR]JMH [>T J<QB >JC] W BJTW B>W,,1.0


In [27]:
dir_he_cases = list(set(df[df.scroll == "MT"].dir_he))
#preps.remove("")
dir_he_cases

['', 1.0, 0.0]

In [28]:
greek_lex_dir_he_prep = {}

for dir_he_case in set(df[df.scroll == "MT"].dir_he):
    heb_verb_ids = df[(df.scroll == "MT") & (df.dir_he == dir_he_case)].verb_id.tolist()
    greek_verb_ids = [verb_id.replace("mt", "lxx") for verb_id in heb_verb_ids]
    
    greek_rows = df[df.verb_id.isin(greek_verb_ids)]
    
    preposition_counts = Counter(greek_rows.preposition_1)
    loci_by_prep = defaultdict(list)

    for _, row in greek_rows.iterrows():
        prep = row.preposition_1
        locus = (row.chapter, row.verse_num)
        loci_by_prep[prep].append(locus)
    
    greek_lex_dir_he_prep[dir_he_case] = {
        "counts": dict(preposition_counts),
        "loci": dict(loci_by_prep)
    }

In [29]:
 greek_lex_dir_he_prep

{'': {'counts': {'': 28, 'πρός': 2, 'ἐπὶ': 1, 'εἰς': 4},
  'loci': {'': [(1, 1),
    (2, 16),
    (2, 17),
    (2, 18),
    (5, 15),
    (6, 11),
    (16, 5),
    (16, 22),
    (17, 8),
    (17, 12),
    (18, 12),
    (19, 7),
    (21, 3),
    (22, 12),
    (22, 25),
    (24, 3),
    (34, 35),
    (35, 5),
    (35, 10),
    (35, 21),
    (35, 22),
    (35, 23),
    (35, 24),
    (35, 25),
    (36, 4),
    (36, 6),
    (40, 4),
    (40, 4)],
   'πρός': [(5, 1), (20, 20)],
   'ἐπὶ': [(10, 4)],
   'εἰς': [(15, 17), (35, 24), (35, 27), (36, 3)]}},
 1.0: {'counts': {'εἰς': 7, '': 1, 'ἐκεῖ ἐσώτερον': 1},
  'loci': {'εἰς': [(1, 1),
    (8, 20),
    (15, 23),
    (15, 27),
    (18, 7),
    (33, 8),
    (33, 9)],
   '': [(10, 26)],
   'ἐκεῖ ἐσώτερον': [(26, 33)]}},
 0.0: {'counts': {'πρός': 20,
   'εἰς': 35,
   'ἐναντίον': 2,
   'ἐπὶ': 4,
   'ὀπίσω': 3,
   'ἀνὰ μέσον': 1,
   'εἰς μέσον': 2,
   '': 6,
   'ἐνώπιον': 1,
   'ἐν': 1,
   'ἐκ': 1,
   'ἔναντι': 1,
   'πα

In [30]:
greek_lex_dir_he = {}

for dir_he_case in dir_he_cases:
    heb_verb_ids = df[df.dir_he == dir_he_case].verb_id
    greek_verb_ids = [verb_id.replace("mt", "lxx") for verb_id in heb_verb_ids]
    dir_hes = list(df[df.verb_id.isin(greek_verb_ids)].lex)
    greek_lex_dir_he[dir_he_case] = Counter(dir_hes)

In [31]:
greek_lex_dir_he

{'': Counter({'εἰσέρχομαι': 32,
          'φέρω': 16,
          'παραγίγνομαι': 12,
          'ἔρχομαι': 12,
          'εἰσπορεύω': 10,
          'εἰσάγω': 7,
          'εἰσφέρω': 6,
          'BW>[': 4,
          'ἥκω': 3,
          'ἀναφέρω': 3,
          'ἐξέρχομαι': 2,
          'εἰσφέρω': 2,
          'ἐπάγω': 2,
          'δυσμή': 2,
          'εἰσάγω': 2,
          'εἰμί': 2,
          'εἴσειμι': 2,
          'πορεύω': 1,
          'προσάγω ': 1,
          'προσέρχομαι': 1,
          'ἄγω': 1,
          'συνάπτω': 1,
          'ἐπάγω': 1,
          'τίθημι': 1,
          'προσφέρω': 1,
          'ποιέω': 1}),
 1.0: Counter({'ἔρχομαι': 3,
          'BW>[': 2,
          'εἰσέρχομαι': 2,
          'εἰσπορεύω': 1,
          'παραγίγνομαι': 1,
          'εἰσάγω': 1,
          'εἰσφέρω': 1}),
 0.0: Counter({'BW>[': 32,
          'εἰσέρχομαι': 22,
          'εἰσπορεύω': 9,
          'φέρω': 7,
          'εἰσάγω': 6,
          'ἔρχομαι': 6,
          'παραγίγνομαι': 5,
          'ἥκω'

In [32]:
greek_lex_dir_he_prep = {}

for dir_he_case in dir_he_cases:
    heb_verb_ids = df[df.dir_he == dir_he_case].verb_id
    greek_verb_ids = [verb_id.replace("mt", "lxx") for verb_id in heb_verb_ids]
    dir_hes = list(df[df.verb_id.isin(greek_verb_ids)].preposition_1)
    greek_lex_dir_he_prep[dir_he_case] = Counter(dir_hes)

In [33]:
greek_lex_dir_he_prep

{'': Counter({'εἰς': 46,
          '': 40,
          'πρός': 22,
          'ἐπὶ': 5,
          'ὀπίσω': 3,
          'ἐναντίον': 2,
          'εἰς μέσον': 2,
          'ἀνὰ μέσον': 1,
          'ἐνώπιον': 1,
          'ἐν': 1,
          'ἐκ': 1,
          'ἐκεῖ ἐσώτερον': 1,
          'ἔναντι': 1,
          'παρὰ': 1,
          'κατὰ': 1}),
 1.0: Counter({'εἰς': 7, '': 3, 'ἐκεῖ ἐσώτερον': 1}),
 0.0: Counter({'εἰς': 35,
          '>L': 27,
          'πρός': 20,
          '': 7,
          'ἐπὶ': 4,
          'ὀπίσω': 3,
          'ἐναντίον': 2,
          'B': 2,
          'εἰς μέσον': 2,
          'L': 1,
          '>XR/': 1,
          'ἀνὰ μέσον': 1,
          'ἐνώπιον': 1,
          'ἐν': 1,
          'ἐκ': 1,
          'ἔναντι': 1,
          'παρὰ': 1,
          'κατὰ': 1})}

In [34]:
df[df.dir_he == 1.0]

,level_0,verb_id,lex,scroll,book,chapter,verse_num,gcons_verb,gcons_verse,sign_info,stem,tense,gcons_clause,subject,complement,cmpl_lex,cmpl_translation,dir_he,cmpl_constr,cmpl_nt,cmpl_anim,cmpl_det,cmpl_indiv,cmpl_complex,motion_type,preposition_1,preposition_2,preposition_3,preposition_4,comments,reconstructed_verse,spatial_arg_type,index
0,0,28771_mt,BW>[,MT,Exodus,1,1,B>JM,W >LH CMWT BNJ JFR>L H B>JM MYRJMH >T J<QB >JC W BJTW B>W,,qal,ptca,H B>JM MYRJMH >T J<QB,,MYRJMH,MYRJM/,to Egypt,1.0,dir_he,topo,inanim,det,nmpr,simple,factive,,,,,,,goal,
54,54,32836_mt,BW>[,MT,Exodus,8,20,JB>,W J<F JHWH KN W JB> <RB KBD BJTH PR<H W BJT <BDJW W B KL >RY MYRJM TCXT H >RY M PNJ H <RB,,qal,wayq,W JB> <RB KBD BJTH PR<H W BJT <BDJW W B KL >RY MYRJM,<RB KBD,BJTH PR<H W BJT <BDJW W B KL >RY MYRJM,BJT/ PR<H/ W BJT/ <BD/ W B KL/ >RY/ MYRJM/,in the house of Pharaoh and the house of his servants and in all the land of Egypt,1.0,prep + dir-he,topo,inanim,det,subs,complex,factive,B,,,,,,goal,
66,66,34407_mt,BW>[,MT,Exodus,10,26,B>NW,W GM MQNNW JLK <MNW L> TC>R PRSH KJ MMNW NQX L <BD >T JHWH >LHJNW W >NXNW L> ND< MH N<BD >T JHWH <D B>NW CMH,,qal,infc,<D B>NW CMH,,CMH,CM,there,1.0,dir_he,adv,inanim,det,adv,simple,factive,,,,,,,goal,
118,118,37239_mt,BW>[,MT,Exodus,15,23,JB>W,W JB>W MRTH W L> JKLW L CTT MJM M MRH KJ MRJM HM <L KN QR> CMH MRH,,qal,wayq,W JB>W MRTH,,MRTH,MRH=/,to Marah,1.0,dir-he,topo,inanim,det,nmpr,simple,factive,,,,,,,goal,
120,120,1955219_dss,BW>[,4Q22,Exodus,15,23,JB>W,W JB>W MRTH W L> JKLW L CTT MJM M MRH KJ MRJM HM <L KN QR> CMH MRH,0 0000 0000 1 11 1111 1 111 111 1 111 11 1111 10 00 00 000 111 111,qal,wayq,W JB>W MRTH,,MRTH,MRH=/,to Marah,1.0,dir-he,topo,inanim,det,nmpr,simple,factive,,,,,,W JB>W MRTH [W L> JKLW L CTT MJM M MRH KJ MRJM H]M <L KN QR> [CMH MRH],goal,54.0
121,121,37331_mt,BW>[,MT,Exodus,15,27,JB>W,W JB>W >JLMH W CM CTJM <FRH <JNT MJM W CB<JM TMRJM W JXNW CM <L H MJM,,qal,wayq,W JB>W >JLMH,,>JLMH,>JLM=/,to Elim,1.0,dir-he,topo,inanim,det,nmpr,simple,factive,,,,,,,goal,
143,143,38602_mt,BW>[,MT,Exodus,18,7,JB>W,W JY> MCH L QR>T XTNW W JCTXW W JCQ LW W JC>LW >JC L R<HW L CLWM W JB>W H >HLH,,qal,wayq,W JB>W H >HLH,,H >HLH,H >HL/,to the tent,1.0,dir-he,topo,inanim,det,subs,simple,factive,,,,,,,goal,
145,145,1950587_dss,BW>[,4Q14,Exodus,18,7,JBJ>HW,W JY> MCH L QR>T XTNW W JCTXW W JCQ LW W JC>LW >JC L R<HW L CLWM W JBJ>HW H >HLH,1 111 000 0 0000 0011 0 00001 1 111 11 1 11111 111 1 1111 1 1110 0 000000 0 0000,hif,wayq,W JBJ>HW H >HLH,,H >HLH,H >HL/,to the tent,1.0,dir-he,obj,inanim,det,subs,simple,factive,,,,,,[W JY>] MCH L QR>T XT[NW] W JCTX[W W JCQ LW W JC>LW >JC L R<HW L CLW]M W JBJ>HW H >HLH,,68.0
202,202,43242_mt,BW>[,MT,Exodus,26,33,HB>T,W NTTH >T H PRKT TXT H QRSJM W HB>T CMH M BJT L PRKT >T >RWN H <DWT W HBDJLH H PRKT LKM BJN H QDC W BJN QDC H QDCJM,,hif,perf,W HB>T CMH M BJT L PRKT >T >RWN H <DWT,,CMH,CM,there,1.0,dir-he,adv,inanim,det,adv,simple,factive,,,,,,,goal,
225,225,47467_mt,BW>[,MT,Exodus,33,8,B>W,W HJH K Y>T MCH >L H >HL JQWMW KL H <M W NYBW >JC PTX >HLW W HBJVW >XRJ MCH <D B>W H >HLH,,qal,infc,<D B>W H >HLH,,H >HLH,H >HL/,in the tent,1.0,dir-he,topo,inanim,det,subs,simple,factive,,,,,,,goal,


In [35]:
# Extract loci for each lexeme

lex_info = df.groupby('lex')[['chapter', 'verse_num']].apply(lambda x: ', '.join(f"({ch},{vs})" for ch, vs in zip(x['chapter'], x['verse_num'])))

# Convert to dictionary
lex_info_dict = lex_info.to_dict()

# Print the result
for lex, occurrences in lex_info_dict.items():
    print(f"{lex}: {occurrences}")

BW>[: (1,1), (1,1), (1,1), (1,19), (1,19), (2,10), (2,16), (2,17), (2,18), (2,18), (3,1), (3,9), (3,9), (3,13), (3,13), (3,13), (3,18), (4,6), (4,6), (5,1), (5,1), (5,15), (5,23), (6,8), (6,8), (6,8), (6,11), (7,10), (7,23), (7,26), (7,28), (8,20), (9,1), (10,1), (10,1), (10,3), (10,3), (10,4), (10,26), (11,1), (12,23), (12,25), (13,5), (13,5), (13,5), (13,5), (13,5), (13,5), (13,5), (13,5), (13,5), (13,5), (13,5), (13,11), (13,11), (13,11), (13,11), (13,11), (13,11), (14,16), (14,16), (14,17), (14,17), (14,20), (14,22), (14,23), (14,23), (14,23), (14,23), (14,28), (14,28), (15,17), (15,17), (15,19), (15,23), (15,23), (15,27), (16,1), (16,5), (16,22), (16,35), (16,35), (17,8), (17,12), (18,5), (18,5), (18,5), (18,6), (18,7), (18,7), (18,12), (18,15), (18,16), (18,16), (18,19), (18,22), (18,23), (18,23), (18,26), (19,1), (19,2), (19,4), (19,7), (19,9), (20,20), (20,24), (21,3), (22,8), (22,12), (22,14), (22,25), (23,19), (23,20), (23,23), (23,27), (24,3), (24,18), (25,14), (26,11), (26,

In [36]:
# Extract loci for each preposition_1 in MT

df_heb = df[df.scroll == "MT"]

prep_info = df_heb.groupby('preposition_1')[['chapter', 'verse_num']].apply(lambda x: ', '.join(f"({ch},{vs})" for ch, vs in zip(x['chapter'], x['verse_num'])))

# Convert to dictionary
prep_info_dict = prep_info.to_dict()

# Print the result
for prep, occurrences in prep_info_dict.items():
    if prep == "":
        prep = "no preposition"
    print(f"{prep}: {occurrences}")

no preposition: (1,1), (1,1), (2,16), (2,17), (2,18), (5,1), (5,15), (6,11), (10,4), (10,26), (15,17), (15,23), (15,27), (16,5), (16,22), (17,8), (17,12), (18,7), (18,12), (19,1), (19,2), (19,7), (20,20), (21,3), (22,12), (22,25), (23,19), (24,3), (26,33), (33,8), (33,9), (34,26), (34,35), (35,5), (35,10), (35,21), (35,22), (35,23), (35,24), (35,24), (35,25), (35,27), (36,3), (36,4), (36,6), (40,4), (40,4)
<D: (22,8)
<L: (11,1), (18,23), (32,21), (34,12), (35,22)
>L: (1,19), (2,18), (3,1), (3,9), (3,13), (3,18), (5,23), (6,8), (7,10), (7,23), (7,26), (9,1), (10,1), (10,3), (12,23), (12,25), (13,5), (13,11), (14,23), (16,1), (16,35), (16,35), (18,5), (18,6), (18,15), (18,16), (18,19), (18,22), (18,26), (19,4), (19,9), (20,24), (23,20), (23,23), (28,29), (28,35), (28,43), (29,30), (30,20), (32,2), (32,3), (36,3), (39,33), (40,21), (40,32), (40,35)
>XR/: (14,17), (14,23), (14,28)
B: (4,6), (4,6), (7,28), (8,20), (14,16), (14,22), (14,28), (15,19), (22,14), (23,27), (24,18), (25,14), (26,1

In [121]:
# Extract loci for each preposition_1 in LXX

df_grk = df[df.scroll == "LXX"]

prep_info = df_grk.groupby('preposition_1')[['chapter', 'verse_num']].apply(lambda x: ', '.join(f"({ch},{vs})" for ch, vs in zip(x['chapter'], x['verse_num'])))

# Convert to dictionary
prep_info_dict = prep_info.to_dict()

# Print the result
for prep, occurrences in prep_info_dict.items():
    if prep == "":
        prep = "no preposition"
    print(f"{prep}: {occurrences}")

no preposition: (1,1), (2,16), (2,17), (2,18), (5,15), (6,11), (10,26), (15,19), (16,5), (16,22), (17,8), (17,12), (18,12), (19,4), (19,7), (21,3), (22,12), (22,14), (22,25), (24,3), (34,35), (35,5), (35,10), (35,21), (35,22), (35,23), (35,24), (35,25), (35,29), (35,29), (36,3), (36,4), (36,6), (40,4), (40,4), (38,24)
ἀνὰ μέσον: (14,20)
ἔναντι: (34,34)
ἐκ: (26,11)
ἐκεῖ ἐσώτερον: (26,33)
ἐν: (25,14)
ἐναντίον: (7,10), (10,3)
ἐνώπιον: (22,8)
ἐπὶ: (10,4), (11,1), (18,22), (18,26), (32,21)
εἰς μέσον: (14,22), (14,23)
εἰς: (1,1), (3,1), (4,6), (4,6), (6,8), (7,23), (7,28), (8,20), (12,23), (12,25), (13,5), (13,11), (14,16), (14,28), (15,17), (15,23), (15,27), (16,1), (16,35), (16,35), (18,7), (18,23), (19,1), (19,2), (23,19), (23,20), (23,27), (24,18), (27,7), (28,29), (28,30), (28,35), (28,43), (29,30), (30,20), (33,8), (33,9), (34,12), (34,26), (35,21), (35,24), (35,27), (36,3), (38,27), (40,21), (40,35)
κατὰ: (36,5)
ὀπίσω: (14,17), (14,23), (14,28)
παρὰ: (35,22)
προ

In [122]:
directive_values = [1.0, 0.0, ""]  # assumes 1.0 and 0.0 are floats, empty is string
greek_lex_directive = {}

for directive in directive_values:
    # Select Hebrew rows with this directive_he value
    heb_rows = df[(df.scroll == "MT") & (df.dir_he == directive)]
    heb_verb_ids = heb_rows.verb_id.astype(str)

    # Convert Hebrew verb_ids to Greek equivalents
    greek_verb_ids = [verb_id.replace("_mt", "_lxx") for verb_id in heb_verb_ids]

    # Select corresponding Greek rows
    df_greek = df[(df.scroll == "LXX") & (df.verb_id.isin(greek_verb_ids))]

    # Extract Greek lexemes and their loci
    lexemes = list(df_greek.lex)
    loci = list(zip(df_greek.chapter, df_greek.verse_num))
    prepositions = list(df_greek.preposition_1)

    # Build dictionary of lexeme → list of loci
    lex_dict = {}
    for lex, locus in zip(lexemes, loci):
        if lex not in lex_dict:
            lex_dict[lex] = []
        lex_dict[lex].append(locus)

    # Optional: add frequency to key
    summary_dict = {f"{lex}, {len(loci)} occ.": loci for lex, loci in lex_dict.items()}
    greek_lex_directive[str(directive)] = summary_dict

In [123]:
greek_lex_directive

{'1.0': {'εἰσπορεύω, 1 occ.': [(1, 1)],
  'παραγίγνομαι, 1 occ.': [(8, 20)],
  'ἔρχομαι, 3 occ.': [(10, 26), (15, 23), (15, 27)],
  'εἰσάγω, 1 occ.': [(18, 7)],
  'εἰσφέρω, 1 occ.': [(26, 33)],
  'εἰσέρχομαι, 2 occ.': [(33, 8), (33, 9)]},
 '0.0': {'εἰσέρχομαι, 22 occ.': [(1, 19),
   (3, 18),
   (7, 10),
   (7, 23),
   (7, 26),
   (7, 28),
   (9, 1),
   (10, 1),
   (10, 3),
   (12, 23),
   (12, 25),
   (14, 16),
   (14, 17),
   (14, 20),
   (14, 22),
   (14, 23),
   (14, 23),
   (15, 19),
   (24, 18),
   (29, 30),
   (35, 29),
   (40, 35)],
  'εἰσάγω, 6 occ.': [(2, 10), (6, 8), (13, 5), (13, 11), (23, 20), (23, 23)],
  'παραγίγνομαι, 5 occ.': [(2, 18), (16, 35), (18, 6), (18, 15), (19, 9)],
  'ἔρχομαι, 6 occ.': [(3, 1), (16, 1), (16, 35), (18, 16), (19, 1), (19, 2)],
  'ἥκω, 3 occ.': [(3, 9), (18, 23), (20, 24)],
  'ἐξέρχομαι, 2 occ.': [(3, 13), (18, 5)],
  'εἰσφέρω, 2 occ.': [(4, 6), (4, 6)],
  'πορεύω, 1 occ.': [(5, 23)],
  'ἐπάγω, 1 occ.': [(11, 1)],
  'εἰσπορεύω, 9 occ.': [(14, 28